In [1]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0-rc2


In [0]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

keras = tf.keras
layers = tf.keras.layers

# 1. get dataset on memory

In [3]:
dir(tf.keras.datasets)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'boston_housing',
 'cifar10',
 'cifar100',
 'fashion_mnist',
 'imdb',
 'mnist',
 'reuters']

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
#255로 나눠준 이유는 정규화 해주는 것
#pixel 값이 255단위로 있기 때문

In [10]:
print("The shape of train dataset : ", x_train.shape)
print("The shape of test dataset : ", x_test.shape)

The shape of train dataset :  (60000, 28, 28)
The shape of test dataset :  (10000, 28, 28)


In [0]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

print("The shape of train dataset : ", x_train.shape)
print("The shape of test dataset : ", x_test.shape)

# 2. Use tf.data to define a data-pipeline

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32).repeat()
#shuffle 10000짜리 Queue로 섞는다 
#repeat epoch를 의미
#batch 32장
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 3. Build the tf.keras model using the Keras model subclassing API:

In [0]:
class MyModel(Model): # inherit Model class of the tf.keras
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.d2 = Dense(num_classes, activation='softmax')

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.flatten(x)
        if training:
          x = self.dropout(x, training=training)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel(num_classes=10)

In [0]:
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Other ways to build your models

### - keras sequential API

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

### - keras functional API

In [0]:
inputs = tf.keras.Input(shape=(32,))  # Returns a placeholder tensor
# 나의 input 차원을 알려준다
# A layer instance is callable on a tensor, and returns a tensor.
x = Dense(64, activation='relu')(inputs)
# x = Denst(128,activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

In [0]:
def get_model():
    inputs = keras.Input(shape=(128,))
    outputs = layers.Dense(1, activation='sigmoid')(inputs)
    return keras.Model(inputs, outputs)

model1 = get_model()
model2 = get_model()
model3 = get_model()

inputs = keras.Input(shape=(128,))
y1 = model1(inputs)
y2 = model2(inputs)
y3 = model3(inputs)
outputs = layers.average([y1, y2, y3])
ensemble_model = keras.Model(inputs=inputs, outputs=outputs)

# 4. define parameters for optimization

## 4.1 use model.fit for simplicity of training

In [20]:
# The compile step specifies the training configuration.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

# Trains for 5 epochs.
import math
steps_per_epoch=math.ceil(60000/32)
model.fit(train_ds, epochs=5, steps_per_epoch=steps_per_epoch)
# Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.

Train for 1875 steps
Epoch 1/5
   1/1875 [..............................] - ETA: 12:21

ValueError: ignored

## 4.2 Use tf.GradientTape for flexibility of training

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
# bce = tf.keras.losses.BinaryCrossentropy()
# loss = bce([0., 0., 1., 1.], [1., 1., 1., 0.]) # tensorflow.python.framework.ops.EagerTensor
# print('Loss: ', loss.numpy())  # Loss: 11.522857
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

# 5. Save and Load your model

In [0]:
# Save weights to a TensorFlow Checkpoint file
model.save_weights('./weights/my_model')

# Restore the model's state,
# this requires a model with the same architecture.
model.load_weights('./weights/my_model')

### Manually save weights

In [0]:
'''

# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

'''

### Save the entire model

to save the a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js (HDF5, Saved Model) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite (HDF5, Saved Model)

In [0]:
'''

# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model shuold be saved to HDF5.
model.save('my_model.h5') 

# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()
'''

In [0]:
import gc
tf.keras.backend.clear_session()
gc.collect()
del model

In [0]:
tf.keras.backend.clear_session()

In [0]:
i = 0
for images, labels in train_ds:
    print(labels)
    #print(labels.numpy())
    i=i+1
    if i==5:
        break